In [3]:
""" Auto Encoder Example.
Build a 2 layers auto-encoder with TensorFlow to compress images to a
lower latent space and then reconstruct them.
References:
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. "Gradient-based
    learning applied to document recognition." Proceedings of the IEEE,
    86(11):2278-2324, November 1998.
Links:
    [MNIST Dataset] http://yann.lecun.com/exdb/mnist/
Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


C:\Users\zhaon\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
X_train = mnist.train.images
y_train = mnist.train.labels
print(type(X_train))
## make sure you only have one notebok at a time

#some variable definitions
batch_size = 1024;
X_batch = X_train[0:batch_size,:]
print('shapes')
print(X_batch.shape)
y_batch = y_train[0:batch_size];
print(y_batch.shape)
#step 1): define an architecture or graph

# step 2): run the graph
tf.reset_default_graph()
# trian the graph with loss
X = tf.placeholder(tf.float32, name = 'X')
y = tf.placeholder(tf.float32);

#without defining a model, we initialize some weights
W = tf.Variable(tf.truncated_normal([256, 784]))
b = tf.Variable(tf.zeros([256, 1]))

W2 = tf.Variable(tf.truncated_normal([10, 256]))
b2 = tf.Variable(tf.zeros([10, 1]))

#propagate
Z1 = tf.matmul(W,X)+b
A1 = tf.nn.sigmoid(Z1);
Z2 = tf.matmul(W2, A1)+b2; #logits
scores = tf.nn.softmax(Z2);
loss = tf.nn.softmax_cross_entropy_with_logits(
    labels=y_batch,
    logits=tf.transpose(Z2),
    dim=-1,
    name='loss'
)
loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-2)
optimizer = optimizer.minimize(loss);
#now try to put an optimizer here

init = tf.global_variables_initializer();
sess = tf.Session();
sess.run(init);
for i in range(100):
    test = sess.run(optimizer, feed_dict={X: X_batch.T})
    if(i%10 == 0):
        print(sess.run(loss, feed_dict={X: X_batch.T}))

sess.close()

<class 'numpy.ndarray'>
shapes
(1024, 784)
(1024, 10)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

10.708355
1.1502838
0.30572996
0.086088344
0.023441363
0.008701508
0.004957712
0.0036333422
0.0029502627
0.0025310358


In [5]:
## define model by layers
tf.reset_default_graph()


input_layer = tf.placeholder(tf.float32, shape = [batch_size, 784])
print(input_layer.shape)
Z1 = tf.layers.dense(input_layer, 100, activation = tf.nn.relu)
Z2 = tf.layers.dense(Z1, 50, activation = tf.nn.relu);
Z3 = tf.layers.dense(Z1, 10, activation = tf.nn.relu);
loss = tf.nn.softmax_cross_entropy_with_logits(
    labels=y_batch,
    logits=Z3,
    dim=-1,
    name='loss'
)
loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-2)
optimizer = optimizer.minimize(loss);

init = tf.global_variables_initializer(); #this initializes ALL VARIABLES
sess = tf.Session();
sess.run(init);

for i in range(100):
    y = sess.run(optimizer, feed_dict={input_layer: X_batch});
    if(i%10 == 0):
        print(sess.run(loss, feed_dict={input_layer: X_batch}))



(1024, 784)
1.9238572
0.56602705
0.37190846
0.3034504
0.27694622
0.26618937
0.26188642
0.2590797
0.25680742
0.25611404


In [19]:
 ## now we encode a layer-based model INTO A FUNCTION

def first_prop(X):
    '''
    we will need to run the session within the function to
    generate the output...which we can feed elsewhere
    '''
    Z1 = tf.layers.dense(X,  100, activation = tf.nn.relu)
    Z2 = tf.layers.dense(Z1, 50, activation = tf.nn.relu);
    Z3 = tf.layers.dense(Z1, 10, activation = tf.nn.relu);

    return Z3; #return an intermediate layer
    
def second_prop(input_from_first):
    Z1 = tf.layers.dense(input_from_first, 10, activation = tf.nn.relu)
    return Z1; #return an intermediate layer


In [24]:
## now we need to be able to train a model with multiple individual modules
# not sure if starting and closing multiple sessions will make sense;
print(y_batch.shape)
x = tf.placeholder(tf.float32, shape = [None, 784])
y1 = first_prop(x); print(y1)
y2 = second_prop(y1); print(y2)

loss = tf.nn.softmax_cross_entropy_with_logits(
    labels = y_batch,
    logits = y2)
loss = tf.reduce_mean(loss) 
optimizer = tf.train.AdamOptimizer(learning_rate = 1e-2)
optimizer = optimizer.minimize(loss);
init = tf.global_variables_initializer();
sess = tf.Session();
sess.run(init);
y = sess.run(loss, feed_dict = {x: X_batch});

print(y)
for i in range(100):
    y = sess.run(optimizer, feed_dict={x: X_batch});
    if(i%10 == 0):
        print(sess.run(loss, feed_dict={x: X_batch}))

sess.close();



(1024, 10)
Tensor("dense_69/Relu:0", shape=(?, 10), dtype=float32)
Tensor("dense_70/Relu:0", shape=(?, 10), dtype=float32)
2.323842
2.15106
1.0394118
0.6843925
0.48739892
0.3770106
0.32151815
0.29706883
0.28487116
0.2793749
0.27654597
